In [7]:
import os
import math

In [8]:
def getSbatchProduction(n_systems, bh_kick, ns_kick = 550, sigma_kicks = [0.3], job_name='sse_production', launcher_file='sse_production_launcher', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    nodes = max(1, len(sigma_kicks))    
    run_size = math.ceil(n_systems/56)

    n_runs = int(n_systems/run_size)
    
    tasks = n_runs*len(sigma_kicks)
    
    
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {nodes*56}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
module load launcher

# Print information
module list

# Launch MPI code...
"""
    for sigma_kick in sigma_kicks:
        preamble_text += f'mkdir $SCRATCH/supernova_remnant_sse_production/bh_{bh_kick:.1f}_ns_{ns_kick:.1f}_sigma_{sigma_kick:.1f}\n'
        
    preamble_text += f'export LAUNCHER_WORKDIR=$SCRATCH/sbatch_sse_supernova_production/ \n'
    preamble_text += f'export LAUNCHER_JOB_FILE={launcher_file}\n\n'
    preamble_text += '${LAUNCHER_DIR}/paramrun'
    
    return preamble_text



In [9]:
def getSbatch_sse_production_launcher(n_systems, bh_kick = 200, ns_kick = 550, sigma_kicks = [0.3], job_name='sse_production', launcher_file='sse_production_launcher', queue='development', timeout = '02:00:00'):
    n_systems = int(n_systems)
    nodes = max(1, len(sigma_kicks))
    run_size = math.ceil(n_systems/56)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs*len(sigma_kicks)
    
    
    
    
    preamble_text = "" 
    
    for sigma_kick in sigma_kicks:
        preamble_text += '\n\n'
        output_path = f'$SCRATCH/supernova_remnant_sse_production/bh_{bh_kick:.1f}_ns_{ns_kick:.1f}_sigma_{sigma_kick:.1f}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode SSE  --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"
        for n in range(56):
            preamble_text += f"~/supernova_remnant/COMPAS_new/COMPAS/src/COMPAS" + specs + f" --random-seed {int(n*2*run_size)} --output-container run_$LAUNCHER_JID \n"
#             preamble_text += f'## {n}\n\n'
    return preamble_text



In [10]:
n_systems = 1e6
bh_kick = 200.0
ns_kicks = [300.0, 350.0, 400.0, 450.0, 500.0, 550.0, 600.0, 650.0, 700.0]
# ns_kicks = [650.0]
sigma_kicks = [0.1, 0.2, 0.3, 0.4, 0.5]
# sigma_kicks = [0.1, 0.2, 0.3, 0.4]


for ns_kick in ns_kicks:
    job_name = f'sbatch_bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kicks[0]}_{sigma_kicks[-1]}'
    launcher_file = f'launcher_ns_{ns_kick}_sigma_{sigma_kicks[0]}_{sigma_kicks[-1]}'
    
    sbatch_text = getSbatchProduction(n_systems, bh_kick, ns_kick, sigma_kicks=sigma_kicks, job_name=job_name, launcher_file=launcher_file, queue='normal', timeout='1:00:00')
    file1 = open(f"{os.environ['SCRATCH']}/sbatch_sse_supernova_production/{job_name}", "w")  # write mode
    file1.write(sbatch_text)
    file1.close()
                 
    launcher_text = getSbatch_sse_production_launcher(n_systems, bh_kick, ns_kick, sigma_kicks=sigma_kicks, job_name=job_name, launcher_file=launcher_file)
    file2 = open(f"{os.environ['SCRATCH']}/sbatch_sse_supernova_production/{launcher_file}", "w")  # write mode
    file2.write(launcher_text)
    file2.close()
print(sbatch_text)

print(launcher_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J sbatch_bh_200.0_ns_700.0_sigma_0.1_0.5           # Job name
#SBATCH -o sbatch_bh_200.0_ns_700.0_sigma_0.1_0.5.o%j	   # Name of stdout output file
#SBATCH -e sbatch_bh_200.0_ns_700.0_sigma_0.1_0.5.e%j	   # Name of stderr error file
#SBATCH -p normal          # Queue (partition) name
#SBATCH -N 5             # Total # of nodes
#SBATCH -n 280             # Total # of mpi tasks
#SBATCH -t 1:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
module load launcher

# Print information
module list

# Launch MPI code...
mkdir $SCRATCH/supernova_remnant_sse_production/bh_200.0_ns_700.0_sigma_0.1
mkdir $SCRA

In [43]:
def getSbatch(n_systems, bh_kick, ns_kick = 400, sigma_kicks = [0.3], run_size=2500, job_name='sse_evolution', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs*len(sigma_kicks)
    nodes = max(1, math.ceil(tasks/56))
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {tasks}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
"""
    for sigma_kick in sigma_kicks:
        preamble_text+= f'mkdir $WORK/supernova_remnant_production/bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}\n' 

    preamble_text += f"\nfor i in `seq 1 {n_runs}` \ndo \n"
    
    for sigma_kick in sigma_kicks:
        output_path = f'$WORK/supernova_remnant_production/bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode SSE --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"
        preamble_text += f"    ibrun -n 1 ~/supernova_remnant/COMPAS/src/COMPAS" + specs + f" --output-container run_$i & \n"
    
    preamble_text += "done \nwait"
    
    return preamble_text



In [45]:
n_systems = 1e6
bh_kick = 200.0
ns_kicks = [300.0]
sigma_kicks = [0.1, 0.2, 0.3, 0.4, 0.5]

for ns_kick in ns_kicks:
    job_name = f'bh_{bh_kick}_ns_{ns_kick}'
    sbatch_text = getSbatch(n_systems, bh_kick, ns_kick, sigma_kicks=sigma_kicks, run_size = 25000, job_name=job_name, queue='normal', timeout='4:00:00')
    file1 = open(f"../sbatch_supernova/{job_name}", "w")  # write mode
    file1.write(sbatch_text)
    file1.close()
        
print(sbatch_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J bh_200.0_ns_300.0           # Job name
#SBATCH -o bh_200.0_ns_300.0.o%j	   # Name of stdout output file
#SBATCH -e bh_200.0_ns_300.0.e%j	   # Name of stderr error file
#SBATCH -p normal          # Queue (partition) name
#SBATCH -N 4             # Total # of nodes
#SBATCH -n 200             # Total # of mpi tasks
#SBATCH -t 4:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
mkdir $WORK/supernova_remnant_production/bh_200.0_ns_300.0_sigma_0.1
mkdir $WORK/supernova_remnant_production/bh_200.0_ns_300.0_sigma_0.2
mkd

In [40]:
def getSbatchOLD(n_systems, bh_kick, ns_kick, sigma_kick, run_size=10000, job_name='sse_evolution', nodes=None, queue='development'):
    output_path = f'$WORK/supernova_remnant/{job_name}'    
    
    n_systems = int(n_systems)
    run_size = int(run_size)
    n_runs = int(n_systems/run_size)
    
    if nodes is None:
        nodes = int(n_runs/56)+1
    tasks = min(56, n_runs)
            
    print(f"Generating script for {n_systems} stars using {n_runs} runs. \n")
  
    if queue=='development':
        timeout = '02:00:00'
    else:
        timeout = '10:00:00'
        
        
    preamble_text = f"""#!/bin/bash
## COSMIC Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {tasks}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

mkdir {output_path}

# Launch MPI code...
    """
    
    if n_runs<1:
        raise ValueError('Total number of stars is less than run size')    
    
    if n_runs>512:
        raise ValueError('Total number of runs exceeds job limit')  
        
    commands_text = ""
    
    for i in range(n_runs):
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode SSE --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"
        commands_text = commands_text + f"srun --exclusive --ntasks=1 --cpus-per-task=2 ~/supernova_remnant/COMPAS/src/COMPAS" + specs + f" --output-container run_{i} & \n"
    
    commands_text = commands_text + "wait"
    preamble_text += commands_text 
    
    return preamble_text



In [6]:
# Test
n_systems = 1e5
bh_kick = 200.0
ns_kicks = [200.0]
sigma_kicks = [0.1]
for ns_kick in ns_kicks:
    for sigma_kick in sigma_kicks:
        job_name = f'sn_rem_bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}'
        sbatch_text = getSbatchOLD(n_systems, bh_kick, ns_kick, sigma_kick, run_size = 2500, job_name=job_name)
        file1 = open(f"../sbatch_supernova/{job_name}", "w")  # write mode
        file1.write(sbatch_text)
        file1.close()
print(sbatch_text)

Generating script for 100000 stars using 40 runs. 

#!/bin/bash
## COSMIC Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J sn_rem_bh_200.0_ns_200.0_sigma_0.1           # Job name
#SBATCH -o sn_rem_bh_200.0_ns_200.0_sigma_0.1.o%j	   # Name of stdout output file
#SBATCH -e sn_rem_bh_200.0_ns_200.0_sigma_0.1.e%j	   # Name of stderr error file
#SBATCH -p development          # Queue (partition) name
#SBATCH -N 1             # Total # of nodes
#SBATCH -n 40             # Total # of mpi tasks
#SBATCH -t 02:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

mkdir $WORK/supernova_remnant/sn_rem_bh_200.0_ns_200.0_sigma_0.1

# Launch M

In [2]:
n_systems = 1e5
bh_kick = 200.0
ns_kicks = [450.0, 500.0, 550.0, 600.0, 650.0, 700.0, 750.0]
sigma_kicks = [0.01, 0.3, 0.7]
for ns_kick in ns_kicks:
    for sigma_kick in sigma_kicks:
        job_name = f'sn_rem_bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}'
        sbatch_text = getSbatch(n_systems, bh_kick, ns_kick, sigma_kick, job_name=job_name)
        file1 = open(f"../sn_rem_sbatch/{job_name}", "w")  # write mode
        file1.write(sbatch_text)
        file1.close()
print(sbatch_text)

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating script for 100000 stars using 10 runs. 

Generating s

In [7]:
def getDefaultSbatch(n_systems, run_size=10000, job_name='sse_evolution', nodes=None, queue='development'):
    output_path = f'$WORK/supernova_remnant_default/{job_name}'    
    
    n_systems = int(n_systems)
    run_size = int(run_size)
    n_runs = int(n_systems/run_size)
    
    if nodes is None:
        nodes = int(n_runs/56)+1
    tasks = min(56, n_runs)
            
    print(f"Generating script for {n_systems} stars using {n_runs} runs. \n")
  
    if queue=='development':
        timeout = '02:00:00'
    else:
        timeout = '10:00:00'
        
        
    preamble_text = f"""#!/bin/bash
## COSMIC Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {tasks}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

mkdir {output_path}

# Launch MPI code...
    """
    
    if n_runs<1:
        raise ValueError('Total number of stars is less than run size')    
    
    if n_runs>512:
        raise ValueError('Total number of runs exceeds job limit')  
        
    commands_text = ""
    
    for i in range(n_runs):
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode SSE --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL"
        commands_text = commands_text + f"srun -N 1 -n 1 ~/COMPAS/src/COMPAS" + specs + f" --output-container run_{i} & \n"
    
    commands_text = commands_text + "wait"
    preamble_text += commands_text 
    
    return preamble_text



In [8]:
n_systems = 1e5
job_name = f'sn_rem_default'
sbatch_text = getDefaultSbatch(n_systems, job_name=job_name)
file1 = open(f"../sn_rem_sbatch/{job_name}", "w")  # write mode
file1.write(sbatch_text)
file1.close()

Generating script for 100000 stars using 10 runs. 

